In [1]:
cd drive/

/content/drive


In [2]:
cd My\ Drive

/content/drive/My Drive


In [3]:
cd Plate_detect_and_recognize-master/

/content/drive/My Drive/Plate_detect_and_recognize-master


In [4]:
# remove warning message
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


# required library
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from utils import detect_lp
from os.path import splitext,basename
from keras.models import model_from_json
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import glob
import math
import statistics
from operator import itemgetter

In [5]:
def load_model(path):
    try:
        path = splitext(path)[0]
        with open('%s.json' % path, 'r') as json_file:
            model_json = json_file.read()
        model = model_from_json(model_json, custom_objects={})
        model.load_weights('%s.h5' % path)
        print("Loading model successfully...")
        return model
    except Exception as e:
        print(e)

In [6]:
wpod_net_path = "wpod-net.json"
wpod_net = load_model(wpod_net_path)

Loading model successfully...


In [7]:
def preprocess_image(image_path,resize=False):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255
    if resize:
        img = cv2.resize(img, (224,224))
    return img

def get_plate(image_path):
    Dmax = 608
    Dmin = 288
    vehicle = preprocess_image(image_path)
    ratio = float(max(vehicle.shape[:2])) / min(vehicle.shape[:2])
    side = int(ratio * Dmin)
    bound_dim = min(side, Dmax)
    _ , LpImg, _, cor = detect_lp(wpod_net, vehicle, bound_dim, lp_threshold=0.5)
    return vehicle, LpImg, cor

In [8]:
def basic_method(plate_image):
    gray = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(7,7),0)
    
    binary = cv2.threshold(blur, 180, 255,
                         cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
    kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    thre_mor = cv2.morphologyEx(binary, cv2.MORPH_DILATE, kernel3)
    return [plate_image, gray, blur, binary, thre_mor]

def color_method(colorNumber, plate_image):
  possibleImages = []
  b = plate_image.copy()
  b = b[:, :, colorNumber]
  y = math.floor(b.shape[0]/2.4)
  x = math.floor(b.shape[1]/9)
  crop_img = b[y:int(y*2), x:int(x*8.3)]
  flatArray = b.flatten()
  cangeV =  np.percentile(flatArray, [32])
  image = cv2.cvtColor(b, cv2.COLOR_BGR2RGB)
  gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  _, binary = cv2.threshold(gray, cangeV, cangeV, cv2.THRESH_BINARY_INV)


  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  
  blur = cv2.GaussianBlur(gray,(7,7),0)
  
  binary = cv2.threshold(blur, 180, 255,
                        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
  
  kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
  thre_mor = cv2.morphologyEx(binary, cv2.MORPH_DILATE, kernel3)
  return [plate_image, gray, blur, binary, thre_mor]

def visualize_set(plot_image):
  # visualize results    
  fig = plt.figure(figsize=(12,7))
  plt.rcParams.update({"font.size":18})
  grid = gridspec.GridSpec(ncols=2,nrows=3,figure = fig)
  # plot_name = ["plate_image","gray","blur","binary","dilation"]

  for i in range(len(plot_image)):
      fig.add_subplot(grid[i])
      plt.axis(False)
      # plt.title(plot_name[i])
      plt.imshow(plot_image[i])

def getFilteredPlateImages(path):
  returnArray = []
  test_image_path = path
  vehicle, LpImg,cor = get_plate(test_image_path)

  if (len(LpImg)): #check if there is at least one license image
    # Scales, calculates absolute values, and converts the result to 8-bit.    
    plate_image = cv2.convertScaleAbs(LpImg[0], alpha=(255.0))
    returnArray.append(basic_method(plate_image))
    for i in range(3):
      returnArray.append(color_method(i,plate_image))
      
  return returnArray

# filteredPlateImages =  getFilteredPlateImages("Plate_examples/50States.jpg")
# filteredPlateImages =  getFilteredPlateImages("Plate_examples/germany_car_plate.jpg")
filteredPlateImages =  getFilteredPlateImages("Plate_examples/photo2.JPG")

error: ignored

In [ ]:
def sort_contours(cnts,reverse = False):
    i = 0
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b: b[1][i], reverse=reverse))
    return cnts

def cutOutliers(inArray):
  returnArray = []
  reducedArray = [((sec[0]+sec[1])/2+(sec[2]+sec[3])/2)/2 for sec in inArray]
  mean = statistics.mean(reducedArray)
  stdevi = statistics.stdev(reducedArray)
  for i in range(len(inArray)):
    if mean-stdevi < reducedArray[i] < mean+stdevi:
      returnArray.append(inArray[i])
  return returnArray

def cutOutliersArea(inArray):
  returnArray = []
  reducedArray = [((con[1] - con[0]) * (con[3] - con[2])) for con in inArray]
  mean = statistics.mean(reducedArray)
  stdevi = statistics.stdev(reducedArray)
  for i in range(len(inArray)):
    if mean-(stdevi*1.5) < reducedArray[i] < mean+(stdevi*1.5):
      returnArray.append(inArray[i])
  return returnArray

def fineChopLetters(contours):

  contours = cutOutliersArea(contours)
  heightList = []
  go = True
  rScore = .05
  passing = 0
  while(rScore < .9 and passing < 6):
    heightList = []
    heightList = [con[1] - con[0] for con in contours]
    average = np.mean(heightList)
    # if abs(rScore) < .85:
    heightDifList = [abs(cow - average) for cow in heightList]
    indexToRemove = np.argmax(heightDifList)
    xPointOfInterest = np.array([(contours[indexToRemove][2] + contours[indexToRemove][3]) / 2])
    xPointOfInterest = xPointOfInterest.reshape(1,-1)
    tempContorCopy = contours.copy()
    del tempContorCopy[indexToRemove]
    bottomEdge = [((con[2] + con[3])/2, con[0]) for con in tempContorCopy ]
    topEdge = [((con[2] + con[3])/2, con[1]) for con in tempContorCopy ]

    X = [((con[2] + con[3])/2) for con in tempContorCopy ]
    X = np.array(X).reshape((-1, 1))

    model = LinearRegression()

    bottomEdgeY = np.array([con[0] for con in tempContorCopy ])
    model.fit(X, bottomEdgeY)
    newYValue = model.predict(xPointOfInterest)
    contours[indexToRemove][0] = int(newYValue)
    rSq1 = model.score(X, bottomEdgeY)

    topEdgeY = np.array([con[1] for con in tempContorCopy ])
    model.fit(X, topEdgeY)
    newYValue = model.predict(xPointOfInterest)
    contours[indexToRemove][1] = int(newYValue)
    rSq2 = model.score(X, topEdgeY)
    rScore = (rSq1 + rSq2) / 2
    passing += 1
    
  return contours

def splitContour(con):
  midPoint = int((con[3] + con[2])/2)
  con1 = con.copy()
  con1[2] = midPoint
  con2 = con.copy()
  con2[3] = midPoint
  return [con1,con2]

def getNeighbors(one,two):

  if one[1] - one[0] > 10 and two[1] - two[0] > 10:
    if (one[1] - one[0]) - (two[1] - two[0]) < 10:
      if abs(one[0] - two[0]) < 5 or abs(one[1] - two[1]) < 5:
        if abs(one[2]-two[3]) < 3 or abs(one[3]-two[2]) < 3:
          return [max(one[0],two[0]), min(one[1],two[1]), min(one[2],two[2]), max(one[3],two[3])]
  return None

def fineChopSmallLetters(contours):
  returnContours = []
  finalReturnContours = []
  if len(contours) == 6:

    processingContours = contours.copy()

    potentialCombine = []

    while (processingContours):
      tempContour = processingContours.pop()
      width = tempContour[3] - tempContour[2]
      height = tempContour[1] - tempContour[0]
      if width/height > 1:
        processingContours = processingContours + splitContour(tempContour)
      elif height/width > 1.7:
        potentialCombine.append(tempContour)
      else:
        returnContours.append(tempContour)
  
    if len(potentialCombine) > 1:

      left = [div[2] for div in potentialCombine]
      right = [div[3] for div in potentialCombine]
      
      for i in range(len(left)-1):
        go = True
        for two in potentialCombine[i+1:]:
          temp = getNeighbors(potentialCombine[i],two)
          if temp != None:
            returnContours.append(temp)
            go = False
        if go:
          returnContours.append(potentialCombine[i])
    elif len(potentialCombine) == 1:
      returnContours.append(potentialCombine[0])
  if len(returnContours) > 2:
    returnContours = cutOutliers(returnContours)
    returnContours = sorted(returnContours, key=itemgetter(2))
  return returnContours


def getContorSets(filteredImages):

  contour_sets = []

  for imageSet in filteredImages:
    
    plate_image = imageSet[0]

    binary = imageSet[3]
    thre_mor = imageSet[4]

    cont, _  = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # creat a copy version "test_roi" of plat_image to draw bounding box
    test_roi_small_l = plate_image.copy()
    test_roi = plate_image.copy()

    # Initialize a list which will be used to append charater image
    crop_characters = []
    small_letters = []

    # define standard width and height of character
    digit_w, digit_h = 30, 60

    totalArea = plate_image.shape[0] * plate_image.shape[1]
    # print(plate_image)


    small_letters_list_top = []
    small_letters_list_bottom = []
    
    for c in sort_contours(cont):
        (x, y, w, h) = cv2.boundingRect(c)
        ratio = h/w
        contorArea = x*h
        if (x + w) < plate_image.shape[1] * .95: #only select if width is relatively small
          if y + h < plate_image.shape[0] *.45: #only select contor not in middle
            small_letters_list_top.append([y,y+h,x,x+w])
          if y > plate_image.shape[0] *.55:
            small_letters_list_bottom.append([y,y+h,x,x+w])

    if(len(small_letters_list_top) > 1):
      small_letters_list_top = fineChopSmallLetters(small_letters_list_top)
      for curr in small_letters_list_top:
        cv2.rectangle(test_roi_small_l, (curr[2], curr[0]), (curr[3], curr[1]), (0, 255,0), 2)
        curr_num = thre_mor[curr[0]:curr[1],curr[2]:curr[3]]
        curr_num = cv2.resize(curr_num, dsize=(digit_w, digit_h))
        _, curr_num = cv2.threshold(curr_num, 220, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        small_letters.append(curr_num)
    count = 0
    curr_num_list = []
    for c in sort_contours(cont):
        # cout = count + 1
        # if count == 2
        (x, y, w, h) = cv2.boundingRect(c)
        ratio = h/w
        if .9<=ratio<=4: # Only select contour with defined ratio
          if y < plate_image.shape[0] *.6 and y + h > plate_image.shape[0] *.4: 
            curr_num_list.append([y,y+h,x,x+w])

    if(len(curr_num_list) > 1):
      curr_num_list = fineChopLetters(curr_num_list)
      for curr in curr_num_list:
        cv2.rectangle(test_roi, (curr[2], curr[0]), (curr[3], curr[1]), (0, 255,0), 2)
        curr_num = thre_mor[curr[0]:curr[1],curr[2]:curr[3]]
        curr_num = cv2.resize(curr_num, dsize=(digit_w, digit_h))
        _, curr_num = cv2.threshold(curr_num, 220, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        crop_characters.append(curr_num)
      print("Detect {} letters...".format(len(crop_characters)))
      fig = plt.figure(figsize=(10,6))
      plt.axis(False)
      # plt.imshow(test_roi)
      plt.imshow(test_roi_small_l)
      # return curr_num_list
      contour_sets.append(small_letters)
      contour_sets.append(crop_characters)
  return contour_sets

crop_characters_set = getContorSets(filteredPlateImages)


In [ ]:
json_file = open('MobileNets_character_recognition.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("License_character_recognition_weight.h5")

labels = LabelEncoder()
labels.classes_ = np.load('license_character_classes.npy')


In [ ]:
def predict_from_model(image,model,labels):
    plt.imshow(image)
    # return
    image = cv2.resize(image,(80,80))
    image = np.stack((image,)*3, axis=-1)
    prediction = labels.inverse_transform([np.argmax(model.predict(image[np.newaxis,:]))])
    return prediction

def read_letters(crop_characters_set):
  for chop_characters2 in crop_characters_set:
    fig = plt.figure(figsize=(15,3))
    cols = len(chop_characters2)
    grid = gridspec.GridSpec(ncols=cols,nrows=1,figure=fig)

    final_string = ''
    for i,character in enumerate(chop_characters2):
        fig.add_subplot(grid[i])
        title = np.array2string(predict_from_model(character,model,labels))
        plt.title('{}'.format(title.strip("'[]"),fontsize=20))
        final_string+=title.strip("'[]")
        plt.axis(False)
        plt.imshow(character,cmap='gray')

    print(final_string)
read_letters(crop_characters_set)